In [ ]:
PROJECT_PATH = "/Users/stephenbo/JOB/PROJECT/quantification_new/"
import sys, os
sys.path.append(PROJECT_PATH + "./utils")
import mysql as mq 
from tools import *
import time
from joinquant import *
import pandas as pd

In [ ]:
mysql = mq.MY_SQL()
jqdata = joinquant("account6")

In [ ]:
select_sql = open("/Users/stephenbo/JOB/PROJECT/quantification_new/sql/get_stock_info.sql").read()
print(select_sql)

In [ ]:
stock_info = mysql.select_table(select_sql)

In [ ]:
mysql.create_table(
    table_name = "stock_daily_price_tmp"#"stock_daily_price"
    ,table_elements = {
        "stock_code":       "VARCHAR(1000)"
        ,"ds":              "VARCHAR(1000)"
        ,"open":            "FLOAT"
        ,"close":           "FLOAT"
        ,"high":            "FLOAT"
        ,"low":             "FLOAT"
        ,"volume":          "FLOAT"
        ,"money":           "FLOAT"
    }
)

In [ ]:
stock_info[:4]

In [ ]:
stock_start_date_dict = dict()
stock_end_date_dict = dict()
stock_code_list = list()
for element in stock_info:
    stock_code = element[1]
    start_date = element[2]
    end_date = element[3]
    stock_start_date_dict[stock_code] = start_date 
    stock_end_date_dict[stock_code] = end_date 
    stock_code_list.append(stock_code)

In [ ]:
stock_daily_price = None
for stock_code in stock_code_list:
    start_date = string_date_convert(stock_start_date_dict[stock_code])
    end_date = "2021-12-16 23:59:59"#get_current_time()
    if start_date != "2021-12-16 00:00:00":
        print("Processing stock_code=%s;start_date=%s;end_date=%s ..." % (stock_code, start_date, end_date))
        stock_daily_price = jqdata.get_daily_price(
            stock_code
            ,start_date = start_date
            ,end_date = end_date
        )
        stock_daily_price['open'] = stock_daily_price['open'].fillna(-1.)
        stock_daily_price['close'] = stock_daily_price['close'].fillna(-1.)
        stock_daily_price['high'] = stock_daily_price['high'].fillna(-1.)
        stock_daily_price['low'] = stock_daily_price['low'].fillna(-1.)
        stock_daily_price['volume'] = stock_daily_price['volume'].fillna(-1.)
        stock_daily_price['money'] = stock_daily_price['money'].fillna(-1.)
        stock_daily_price_elements = mysql.get_insert_elements(stock_daily_price, stock_daily_price.columns)
        mysql.insert_table(
            table_name = "stock_daily_price_tmp"
            ,attribute_list = stock_daily_price.columns
            ,elements = stock_daily_price_elements
        )
        time.sleep(1)
stock_daily_price